## Import thư viện

In [1]:
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, when, to_timestamp, split, regexp_replace, row_number, sum, count_distinct
from functools import reduce
import pyspark.pandas as ps

C:\Users\solit\AppData\Roaming\Python\Python311\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Khởi tạo Spark Session

In [2]:
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

## Đọc file data

In [3]:
raw_df = spark.read.csv("./data/raw_data.csv", header=True, inferSchema=True)

In [4]:
raw_df.show()

+-----------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|    category_id|        publish_time|                tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|Jw1Y-zhQURU|     17.14.11|John Lewis Christ...|          John Lewis|  Howto & Style|2017-11-10T07:38:...|"christmas|""john...| 7224515|  55681|   10247|         9479|https://i.ytimg.c...|        

In [5]:
raw_df.describe().show()

+-------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------------+----------------+----------------------+--------------------+
|summary|   video_id|trending_date|               title|       channel_title|         category_id|        publish_time|                tags|               views|               likes|            dislikes|    comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------------+----------------+----------------------+--------------------+
|  count|      43185|        39540

In [6]:
def dataframe_info(df):
    print(f"{'-'*40}")
    print(f"DataFrame thông tin:")
    print(f"Số dòng: {df.count()}")
    print(f"Số cột: {len(df.columns)}")
    print(f"{'-'*40}")
    print("Schema:")
    df.printSchema()
    print(f"{'-'*40}")
    print("Số giá trị null trong mỗi cột:")
    null_counts = df.select([
        count(when(col(c).isNull(), c)).alias(c) for c in df.columns
    ])
    null_counts.show()

In [7]:
dataframe_info(raw_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 43295
Số cột: 16
----------------------------------------
Schema:
root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

----------------------------------------
Số giá trị null trong mỗi cột:
+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-

## Tiền xử lí dữ liệu

### Xóa các cột không cần thiết

In [8]:
category_df = raw_df.drop(*['video_id', 'thumbnail_link', 'comments_disabled', 'video_error_or_removed', 'ratings_disabled'])
dataframe_info(category_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 43295
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = true)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-------

### Xóa các hàng có tất cả các giá trị là Null

In [9]:
category_df = category_df.filter(
    reduce(lambda a, b: a | b, (col(c).isNotNull() for c in category_df.columns))
)
category_df.show()

+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+
|trending_date|               title|       channel_title|    category_id|        publish_time|                tags|   views|  likes|dislikes|comment_count|         description|
+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+
|     17.14.11|John Lewis Christ...|          John Lewis|  Howto & Style|2017-11-10T07:38:...|"christmas|""john...| 7224515|  55681|   10247|         9479|Click here to con...|
|     17.14.11|Taylor Swift: …Re...| Saturday Night Live|  Entertainment|2017-11-12T06:24:...|"SNL|""Saturday N...| 1053632|  25561|    2294|         2757|Musical guest Tay...|
|     17.14.11|Eminem - Walk On ...|          EminemVEVO|          Music|2017-11-10T17:00:...|"Eminem|""Walk""|...|

In [10]:
dataframe_info(category_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 39540
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = true)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-------

### Xóa các hàng có trending_date sai định dạng (lỗi data => các giá trị khác trong hàng có nhiều giá trị Null)

In [11]:
category_df = category_df.filter(
    col("trending_date").rlike(r"^\d{2}\.\d{2}\.\d{2}$")
)
dataframe_info(category_df)


----------------------------------------
DataFrame thông tin:
Số dòng: 38806
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = true)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-------

### Thêm giá trị cho các hàng có cột description có giá trị bằng Null

In [12]:
category_df = category_df.fillna({"description": "No description"})
dataframe_info(category_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 38806
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = false)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+------

### Chuẩn hóa dữ liệu

In [13]:
category_df = category_df.withColumn('trending_date', to_timestamp('trending_date', 'yy.dd.MM'))
category_df = category_df.withColumn('publish_time', to_timestamp('publish_time', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
category_df = category_df.withColumn('tags', when(category_df['tags'] == '[none]', '').otherwise(category_df['tags']))
category_df = category_df.withColumn('tags', split(regexp_replace("tags", '"', ""), "\\|"))
category_df.show()

+-------------------+--------------------+--------------------+---------------+-------------------+--------------------+--------+-------+--------+-------------+--------------------+
|      trending_date|               title|       channel_title|    category_id|       publish_time|                tags|   views|  likes|dislikes|comment_count|         description|
+-------------------+--------------------+--------------------+---------------+-------------------+--------------------+--------+-------+--------+-------------+--------------------+
|2017-11-14 00:00:00|John Lewis Christ...|          John Lewis|  Howto & Style|2017-11-10 07:38:29|[christmas, john ...| 7224515|  55681|   10247|         9479|Click here to con...|
|2017-11-14 00:00:00|Taylor Swift: …Re...| Saturday Night Live|  Entertainment|2017-11-12 06:24:44|[SNL, Saturday Ni...| 1053632|  25561|    2294|         2757|Musical guest Tay...|
|2017-11-14 00:00:00|Eminem - Walk On ...|          EminemVEVO|          Music|2017-11-10 

In [14]:
dataframe_info(category_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 38806
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = false)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+-----

## Phân tích

### Số category

In [15]:
category_df.select('category_id').distinct().show()

+--------------------+
|         category_id|
+--------------------+
|           Education|
|              Gaming|
|       Entertainment|
|     Travel & Events|
|Science & Technology|
|              Sports|
|       Howto & Style|
|    Film & Animation|
|      People & Blogs|
|     News & Politics|
|      Pets & Animals|
|    Autos & Vehicles|
|               Music|
|              Comedy|
+--------------------+



In [16]:
category_df.select('category_id').distinct().count()

14

### Category nằm top trending nhiều nhất (1vid/ngày = 1 lần)

In [17]:
category_df.groupBy('category_id').count().orderBy("count", ascending=False).show()

+--------------------+-----+
|         category_id|count|
+--------------------+-----+
|               Music|13754|
|       Entertainment| 9124|
|      People & Blogs| 2926|
|    Film & Animation| 2577|
|       Howto & Style| 1928|
|              Sports| 1907|
|              Comedy| 1828|
|              Gaming| 1788|
|     News & Politics| 1225|
|      Pets & Animals|  534|
|Science & Technology|  518|
|           Education|  457|
|    Autos & Vehicles|  144|
|     Travel & Events|   96|
+--------------------+-----+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [18]:
pandas_category_df = category_df.pandas_api()
pandas_category_df['category_id'].value_counts().plot.bar()

C:\Users\solit\AppData\Roaming\Python\Python311\site-packages\pyspark\pandas\base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


### Category có tổng số lượng views cao nhất (Tổng số view (cao nhất) của các vid thuộc category cụ thể)

#### Tổng số vid

In [19]:
category_df.select('title').distinct().show()

+--------------------+
|               title|
+--------------------+
|Google Pixelbook:...|
|Fall Out Boy - HO...|
|The Poop In My Pa...|
|I Picked My Girlf...|
|New album Open He...|
|WWE fan interrupt...|
|ASOS UNBOXING HAU...|
|Let's talk... YOU...|
|Parenting Habits ...|
|Charles Manson  -...|
|[OFFICIAL VIDEO] ...|
|Maroon 5 Busks in...|
|BTS interview at ...|
|Vance Joy - Like ...|
|SECRETS REVEALED!...|
|Demi Lovato - Sky...|
|What To Buy HER: ...|
|WE'RE MOVING! | L...|
|Proto-Putty in Li...|
|Little Mix - Glor...|
+--------------------+
only showing top 20 rows



In [20]:
category_df.select('title').distinct().count()

3363

#### Lọc lấy số view cao nhất của từng vid

In [21]:
windowSpec = Window.partitionBy("title").orderBy(col("views").desc())
windowSpec

In [22]:
view_df = category_df.withColumn("view_rank", row_number().over(windowSpec))
view_df.show()

+-------------------+--------------------+-----------------+--------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+---------+
|      trending_date|               title|    channel_title|   category_id|       publish_time|                tags| views|likes|dislikes|comment_count|         description|view_rank|
+-------------------+--------------------+-----------------+--------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+---------+
|2017-11-16 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-11-05 20:10:16|[Ben Cathro, sick...| 16074|  689|       8|          142|How flipping rad ...|        1|
|2017-11-15 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-11-05 20:10:16|[Ben Cathro, sick...| 15818|  686|       8|          141|How flipping rad ...|        2|
|2017-11-14 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-

In [23]:
view_df = view_df.filter(col('view_rank') == 1)
view_df.show()

+-------------------+------------------------+--------------------+----------------+-------------------+-----------------------+--------+------+--------+-------------+--------------------+---------+
|      trending_date|                   title|       channel_title|     category_id|       publish_time|                   tags|   views| likes|dislikes|comment_count|         description|view_rank|
+-------------------+------------------------+--------------------+----------------+-------------------+-----------------------+--------+------+--------+-------------+--------------------+---------+
|2017-11-16 00:00:00|    #21 How to go FAS...|          Ben Cathro|          Sports|2017-11-05 20:10:16|   [Ben Cathro, sick...|   16074|   689|       8|          142|How flipping rad ...|        1|
|2017-11-22 00:00:00|    #VeteransDay: Tha...|   YouTube Spotlight|   Entertainment|2017-11-10 15:07:13|   [vets, veterans d...|  916104| 27405|    2153|         5292|This #VeteransDay...|        1|
|2017

#### Tính tổng số view theo category

In [24]:
total_view_df = view_df.groupBy('category_id').agg(sum('views').alias('total_views')).orderBy("total_views", ascending=False)
total_view_df = total_view_df.withColumn("total_views", col("total_views").cast("long"))
total_view_df.show()

+--------------------+-----------+
|         category_id|total_views|
+--------------------+-----------+
|               Music| 6944183379|
|       Entertainment| 1747446715|
|    Film & Animation|  368732367|
|              Comedy|  327819293|
|      People & Blogs|  304066688|
|              Sports|  196526000|
|              Gaming|  115670660|
|     News & Politics|   85441934|
|       Howto & Style|   82587255|
|Science & Technology|   81132989|
|      Pets & Animals|   29851061|
|           Education|   26473589|
|    Autos & Vehicles|   14738146|
|     Travel & Events|   10627822|
+--------------------+-----------+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [25]:
pandas_total_view_df = total_view_df.pandas_api()
pandas_total_view_df.plot.bar(x='category_id', y='total_views')

### Category có nhiều kênh youtube làm nhất

In [26]:
channel_df = category_df.groupBy('category_id').agg(count_distinct('channel_title').alias('total_channels')).orderBy("total_channels", ascending=False)
channel_df.show()

+--------------------+--------------+
|         category_id|total_channels|
+--------------------+--------------+
|               Music|           505|
|       Entertainment|           339|
|      People & Blogs|           188|
|              Gaming|           124|
|    Film & Animation|           103|
|              Sports|            92|
|       Howto & Style|            82|
|              Comedy|            75|
|     News & Politics|            61|
|Science & Technology|            34|
|           Education|            31|
|      Pets & Animals|            25|
|    Autos & Vehicles|            14|
|     Travel & Events|             9|
+--------------------+--------------+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [27]:
pandas_channel_df = channel_df.pandas_api()
pandas_channel_df.plot.bar(x='category_id', y='total_channels')